In [1]:
import numpy as np

In [2]:
def write_header_lines(file, layer_height, line_width, layer_count, mesh, start_x, start_y, start_z=0.3):
    with open(file, 'w') as f:
        header_lines = [
            f";layer_height = {layer_height}",
            f"\n;line_width = {line_width}",
            f"\n;layer_count = {layer_count}",
            f"\n;mesh = {mesh}"
        ]
        
        initialize_lines = [
            "\nG21 ;start of the code",
            "\nG1 Z15 F300",
            "\nG28 X0 Y0 ;Home",
            "\nG92 X0 Y0 ;Consider this as current",
            "\nG0 X50 Y50 F3000 ;Go-to Offset",  
            "\nG92 X0 Y0 ;Reset",
            "\n",
            f"\nG0 F3600 X{start_x:.3f} Y{start_y:.3f} Z{start_z:.3f} ;Go to start position",
            "\nM7",
            "\n\n"
        ]
        
        f.writelines(header_lines)
        f.writelines(initialize_lines)
        
        
def writ_finish_lines(file):
    with open(file, 'a') as f:
        finish_lines = [
            "\n\n;Finish",
            "\nM9",
            "\nG28 X-100 Y-100;Home",
        ]
        f.writelines(finish_lines)

In [3]:
def write_G1_line(delta_x, delta_y, xy, lines):
    xy += [delta_x, delta_y]
    lines.append(f"\nG1 X{xy[0]:.3f} Y{xy[1]:.3f}")
    # print(xy)
    return xy, lines

def go_to_next_layer(z, xy, lines, layer_height):
    z += layer_height
    lines.append(f"\nG1 Z{z:.3f}")
    return z, lines

In [4]:
def soft_film(file, a, b, w, h, z=0.3):
    xy = np.array([-(a-w)/2, -(b-w)/2])
    lines = []
    
    write_G1_line(-np.sign(xy[0])*(a-w), 0, xy, lines)
    while xy[1] < (b-w)/2:
        xy, lines = write_G1_line(0, w, xy, lines)
        xy, lines = write_G1_line(-np.sign(xy[0])*(a-w), 0, xy, lines)
        
    z, lines = go_to_next_layer(z, xy, lines, h)
        
    write_G1_line(0, -np.sign(xy[1])*(b-w), xy, lines)
    if np.sign(xy[0]) > 0:
        while xy[0] > -(a-w)/2+0.0001:
            xy, lines = write_G1_line(-w, 0, xy, lines)
            xy, lines = write_G1_line(0, -np.sign(xy[1])*(b-w), xy, lines)
        
    else:
        while xy[0] < (a-w)/2-0.0001:
            xy, lines = write_G1_line(w, 0, xy, lines)
            xy, lines = write_G1_line(0, -np.sign(xy[1])*(b-w), xy, lines)

    with open(file, 'a') as f:
        f.writelines(lines)

In [158]:
i = 8
(0.2*(1+i))**2 + 0.2

3.4400000000000004

In [6]:
def base_structure(file, a, b, w, h, z=0.3):
    soft_film(file, a, b, w, h, z)
    
    z = 0.7    
    xy = np.array([(a-w)/2, (b-w)/2])
    lines = []
    
    # draw right side first
    for i in range (10):
        z, lines = go_to_next_layer(z, xy, lines, h)
        
        if not (i % 2):
            boundary = ((i+1)*0.4-0.2)**0.5
            xx = xy[0] - boundary
            print(xx)
            xy, lines = write_G1_line(-xx, 0, xy, lines)
            
            if xy[1] > 0:
                while xy[1] > - (b-w)/2 + 0.0001:
                    xy, lines = write_G1_line(0, -w, xy, lines)
                    xy, lines = write_G1_line(-np.sign(xy[0]-xx/2)*xx, 0, xy, lines)
            
            else:
                while xy[1] < (b-w)/2 - 0.0001:
                    xy, lines = write_G1_line(0, w, xy, lines)
                    xy, lines = write_G1_line(-np.sign(xy[0]-xx/2)*xx, 0, xy, lines)
                    
            if xy[0] > xx/2:
                xy, lines = write_G1_line(-xx, 0, xy, lines)
                    
        else:
            xy, lines = write_G1_line(0, -np.sign(xy[1])*(b-w), xy, lines)
            while xy[0] < (a-w)/2 - 0.0001:
                xy, lines = write_G1_line(w, 0, xy, lines)
                xy, lines = write_G1_line(0, -np.sign(xy[1])*(b-w), xy, lines)
                
    # then the left side
    lines.append("\nM9")
    lines.append(f"\nG1 Z{z+5:.3f}")
    xy = np.array([-(a-w)/2, (b-w)/2])
    lines.append(f"\nG1 X{xy[0]:.3f} Y{xy[1]:.3f}")
    z = 0.7
    lines.append(f"\nG1 Z{z:.3f}")
    lines.append("\nM7")
    
    for i in range (10):
        z, lines = go_to_next_layer(z, xy, lines, h)
        
        if not (i % 2):
            boundary = ((i+1)*0.4-0.2)**0.5
            xx = -xy[0] - boundary
            print(boundary, xx)
            xy, lines = write_G1_line(xx, 0, xy, lines)
            
            if xy[1] > 0:
                while xy[1] > - (b-w)/2 + 0.0001:
                    xy, lines = write_G1_line(0, -w, xy, lines)
                    xy, lines = write_G1_line(np.sign(-xy[0]-xx/2)*xx, 0, xy, lines)
            
            else:
                while xy[1] < (b-w)/2 - 0.0001:
                    xy, lines = write_G1_line(0, w, xy, lines)
                    xy, lines = write_G1_line(np.sign(-xy[0]-xx/2)*xx, 0, xy, lines)
                    
            if xy[0] < -xx/2:
                xy, lines = write_G1_line(xx, 0, xy, lines)
                    
        else:
            xy, lines = write_G1_line(0, -np.sign(xy[1])*(b-w), xy, lines)
            while xy[0] > -(a-w)/2 + 0.0001:
                xy, lines = write_G1_line(-w, 0, xy, lines)
                xy, lines = write_G1_line(0, -np.sign(xy[1])*(b-w), xy, lines)
    
    
    

    with open(file, 'a') as f:
        f.writelines(lines)

In [194]:
layer_height = 0.4
line_width = 0.4
layer_count = 2
mesh = 'base structure'
length_a = 40
length_b = 20
wall_layer_count = 0
file = '7.19/base.gcode'
write_header_lines(file, layer_height, 0.4, layer_count, mesh, -length_a/2, -length_b/2)

base_structure(file, length_a, length_b,line_width, layer_height, z=0.3)
        
writ_finish_lines(file)

19.352786404500044
19.04721359549995
18.458359213500124
18.529189236840157
18.168542658201126
0.4472135954999579 19.352786404500044
1.0 19.04721359549995
1.3416407864998738 18.458359213500124
1.61245154965971 18.529189236840157
1.8439088914585775 18.168542658201126


In [5]:
layer_height = 0.4
line_width = 0.4
layer_count = 2
mesh = 'soft film'
length_a = 20
length_b = 30
wall_layer_count = 0
file = '7.29/film_20_30.gcode'
write_header_lines(file, layer_height, 0.4, layer_count, mesh, -length_a/2, -length_b/2)

soft_film(file, length_a, length_b, line_width, layer_height)
# soft_film(file, length_a, length_b, line_width, layer_height, z=1.1)
        
writ_finish_lines(file)

In [6]:
layer_height = 0.4
line_width = 0.4
layer_count = 2
mesh = 'soft film'
length_a = 30
length_b = 6
wall_layer_count = 0
file = '7.31/film_30_6.gcode'
write_header_lines(file, layer_height, 0.4, layer_count, mesh, -length_a/2, -length_b/2)

soft_film(file, length_a, length_b, line_width, layer_height)
# soft_film(file, length_a, length_b, line_width, layer_height, z=1.1)
        
writ_finish_lines(file)